# Pandas Functions for Data Analysis

1. **apply(), and groupby() with apply()**
2. **nlargest(), nsmallest(), using sum, and mean functions**
3. **boolean mask**
4. **complex analysis using all of the above**   

## The purpose of this notebook is to work through some pandas functions and concepts that are commonly used in data analysis, in a problem-solving format.

## The types of analyses that we cover here are ones that you could possibly be asked to recreate in some fashion, before the semester's end.

In [ ]:
# !wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/nba_stats.csv
# !wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/worst_players.csv
# !wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/best_players.csv
# !wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/top_rebs.csv
# !wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/top_mins.csv 

In [ ]:
# some modules we will need
import pandas as pd
import numpy as np

We will be using some data from the National Basketball Association's (NBA) statistics API for this exercise. The data is from the 2017-2020 seasons and includes the major statistics for players.

We will import the data into a dataframe called nba_stats and take a quick look at the data.

In [ ]:
# load the data file
# bring in the sample output file    
nba_stats = pd.read_csv('nba_stats.csv')
# create df with only the columns we want to work with
nba_stats= nba_stats[['SEASON_ID','PLAYER_ID','PLAYER_NAME','GP','MIN','PTS','REB','PLUS_MINUS']]
nba_stats = nba_stats.rename(columns={"GP": "GAMES_PLAYED", "MIN": "MINUTES","PTS": "POINTS", "REB": "REBOUNDS"})

### Before we get started on the functions, let's take a quick look at some of the key data fields that we will be working with, and some fields whose meaning may not be easily discernble from the name.

- `PLAYER_ID` - The unique ID number for each player.
- `SEASON_ID` - The ID number for each season. The combination of PLAYER_ID and SEASON_ID gives us the primary key for the dataframe.
- `PLAYER_NAME` - The name of each player.

#### Note that there are 2,139 rows in the dataframe. That means we have 2,139 unique player-season combinations.

- `GAMES_PLAYED` through `PLUS_MINUS` columns- The individual statistics for the player for that season. Whenever we are working with one of the columns, we will define what that column means in the exercise.

#### The info() and describe() functions are good to use when first looking at a dataframe.

info() gives us column information, and describe() gives us some statistical measurements of the dataframe.

In [ ]:
nba_stats.info()
nba_stats.describe()

## The apply() function

#### `apply()` is used to apply a function to a data frame or to a series (column of the data frame).

The basic way to use the function is:

out = `dataframe`.apply(`func`)

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

### Use the apply() function on a single column of the dataframe

Pass a built-in function to apply().

What is the average number of games that a player played in during any season?

In [ ]:
# note the syntax of using the DOUBLE BRACKETS around the column name.
mean_value = nba_stats[['GAMES_PLAYED']].apply(np.mean)
print(mean_value)

nba_stats[['GAMES_PLAYED']].apply(np.mean)

#### We can also use the apply function on multiple columns or the entire dataframe, but to do so, all of the dataframe columns must be able to be operated on by the function we are applying. 

#### With this data, we can apply to multiple columns that are INT and FLOAT, but not to the entire dataframe, because we also have OBJECT data types.

What is the average number of games, points scored, and rebounds for the typical player in a season?

In [ ]:
nba_stats[['GAMES_PLAYED','POINTS','REBOUNDS']].apply(np.mean)

# returns value error of "could not convert string to float"
# nba_stats.apply(np.mean)

As you can see, the function returns a value for each column.

That is to say, the default way of apply( ) dealing with a dataframe is to take a whole column each time and operate on that column with the function passed.

We can change this default setting by specifying the `axis` parameter, in which axis=0 (the default) applies by column and axis=1 applies by row. We will not demonstrate row-based apply with this dataset.

### Remember the groupby() function from the last notebook.

A `groupby()` operation involves some combination of splitting the object, applying a function, and combining the results. This can be used to group large amounts of data and compute operations on these groups.

The basic way to use the function is:

out = `dataframe`.groupby(by=columnname).`function`()

For example:

df.groupby(by=["b"]).sum()

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

On this dataset, an example might be:

In [ ]:
nba_stats.groupby(by=["SEASON_ID"]).mean()

#### So what `groupby()` on its own does is a dataframe-wide grouping of every `APPLICABLE` column of the passed-in function, using the "by" parameter that we set. 

#### Note that the `groupby()` above does not include the OBJECT column of PLAYER_NAME.

#### Also note that the column `PLAYER_ID` is included, because it is data type `int64`. 

However, we very seldom want to group an entire df in our analyses. Instead we want to generally:

1. Return statistical analyses for individual or multiple columns
2. Grouped by multiple dimensions (columns).

### So how do we do that? By using `groupby()` and `apply()` together.

**The syntax for a single column looks like:**

`dataframe.groupby('columnname').apply(function)`

**The syntax for a multiple columns looks like:**

`dataframe.groupby(['columnname1','columnname2']).apply(function)`

Remember that using `axis=0` (the default) will apply the given function to each *column* and `axis=1` will apply the given function to each *row*.

While `Series.apply` works on individual values and `DataFrame.apply` works on `Series` objects (rows or columns are instances of `Series`), `groupby.apply` works on `DataFrame` objects. The cell below is applying the `print` function to each `DataFrame` or "group" in the `groupby`!

In [ ]:
nba_stats.groupby('SEASON_ID').apply(np.sum, axis=0)

In [ ]:
nba_stats.groupby('SEASON_ID').apply(np.mean, axis=0)

#### Note the difference in behavior between apply() alone, groupby() alone, and groupby.apply() together. This is important for students to understand!!

1. `apply()` by itself gives us the function result for the columns/rows **IT IS ABLE TO OPERATE ON**. 

    As we saw above, if we try to perform a function on an incompatible column/row, it will return an error.


2. `groupby()` by itself ALSO gives us the function result for the columns/rows **IT IS ABLE TO OPERATE ON**. 

    However, it will **SIMPLY NOT INCLUDE** the columns that the function cannot operate on in, the result set.


3. `groupby()` `apply()` together will return the function result for the columns/rows **IT IS ABLE TO OPERATE ON**, similar to `groupby()` alone. 
    
    However, the difference is with the PLAYER_NAME column, in that the sum() function "added" the player names by concatenating (using "+") the strings together. 
    
    Using np.mean() gave us the result with the "nuisance columns" error message.

## So what is the problem with the above approach? And how can we fix it?

### The problem is that, depending on the function we are using (sum vs. mean, for example), some columns may be included in (or excluded from) the returned data frame. So we may get results that we are not expecting, and the test cases will fail as a result.

### So how do we fix that and prevent it from happening?

`1. Create a new dataframe by keeping only the columns necessary for that particular analysis.`

`2. (Optional) Set your columns to groupby as indices on the new dataframe.(This ensures that you are not grouping extraneous columns, for functions such as sum())`

`3. Perform the required groupby/apply/function on the new dataframe.`

`4. (Optional) Set the index columns to be regular columns.`


#### Below are additional steps that the (exam/homework) exercise may require.


`5. Merge the returned dataframe with the other dataframe(s) required by the analysis.`

`6. Drop the extraneous columns in the new/merged dataframe.`

`7. Rename the remaining columns, per the exercise requirements.`

### So how might we want to use this is in a real (or testing) scenario?

Return a dataframe that summarizes the total minutes, games played, points, and rebounds for each player, over the 4 seasons. 

Use your dataframe, nba_stats, as the starting point.

#### What is our strategy for solving this problem?

1. Create a dataframe with only the columns required.

2. Set the grouping columns to be indexes (optional).

3. Perform groupby and apply for the sum.

4. Set the indexes back to be regular columns.

5. Perform additional steps as required by the exercise requirements.

In [ ]:
# create new dataframe with only the required columns
nba_stats_test = nba_stats[['PLAYER_NAME','GAMES_PLAYED','POINTS','REBOUNDS']]
print(' New dataframe')
print(nba_stats_test.head(5))

# (optional) set the grouping columns to be indexes
nba_stats_test = nba_stats_test.set_index(['PLAYER_NAME'])
print('\n Column as index')
print(nba_stats_test.head(5))

# perform the groupby.apply
nba_stats_test2 = nba_stats_test.groupby('PLAYER_NAME').apply(np.sum, axis=0)
print('\n Grouping')
print(nba_stats_test2.head(5))

# set the index columns back to be regular columns
nba_stats_test2.reset_index(inplace=True)
print('\n Set index back to column')
print(nba_stats_test2)

# perform whatever other steps the analysis requires

#### So what if we did not set the column to be an index? What would happen then? Compare the results below with those above.

In [ ]:
# create new dataframe with only the required columns
nba_stats_test = nba_stats[['PLAYER_NAME','GAMES_PLAYED','POINTS','REBOUNDS']]
print(' New dataframe')
print(nba_stats_test.head(5))

# (optional) set the grouping columns to be indexes
# commented out here
# nba_stats_test = nba_stats_test.set_index(['PLAYER_NAME'])
# print(nba_stats_test.head(5))

# perform the groupby.apply
nba_stats_test2 = nba_stats_test.groupby('PLAYER_NAME').apply(np.sum, axis=0)
print('\n Grouping')
print(nba_stats_test2.head(5))

# set the index columns back to be regular columns
# commented out here
# nba_stats_test2.reset_index(inplace=True)
# nba_stats_test2

# perform whatever other steps the analysis requires

### Some good references

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.SeriesGroupBy.apply.html

https://datagy.io/pandas-groupby/

https://www.geeksforgeeks.org/grouping-and-aggregating-with-pandas/

https://datagy.io/pandas-exploratory-data-analysis/

https://stackabuse.com/efficient-data-manipulation-with-apply-function-in-pandas/

### Any questions up to this point?

### Now let's use some functions to do a more complex analysis, by player and season. What we are about to go over would be typical for a multi-point exercise on an exam.

**We will create a new dataframe, `nba_stats_3`, for this, using the previous dataframe, `nba_stats`. This dataframe will have `PLAYER_NAME` and `SEASON_ID` as the grouping columns, and we will take the optional step to set them as the indices.**

In [ ]:
nba_stats_3 = nba_stats.set_index(['SEASON_ID','PLAYER_NAME'])
print('\n Column as index')
print(nba_stats_3.head(5))

**Requirement**:  

Return a dataframe, top_rebs, containing the player name and season for the top 5 number of rebounds across the 4 seasons. 
    
    Include the top 5 plus ties. In other words, if there are ties, keep all of the results, even if it results in more than 5 rows being returned. 
    
    The dataframe should be sorted from most to least, with ties broken by name in alphabetical order. 

Use the nba_stats_3 dataframe as the input for this. 

The output dataframe should have the following columns:  `player`, `season`, `total_rebounds`.

## Pandas Functions `nlargest()` and `nsmallest()`

To meet this requirement, we will want to use the pandas function:  `nlargest()`. The function `nsmallest()` operates in the same manner, and you would want to use this if the exercise requirement was for the least/lowest number of rows.

We have explicitly stated what function to use here, but on an exam, you might see something like "the pandas function nlargest might be useful for this exercise".

The requirement on ties is satisfied by the parameter "keep", and the value of "all".

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nlargest.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nsmallest.html

#### What is our strategy for solving this problem?

1. Create a dataframe with only the columns required.

2. Return/keep only the top 5 in the dataframe.

3. Set the indexes to be columns.

4. Rename the columns.

5. Sort the dataframe.

In [ ]:
# create the dataframe with only the columns required
top_rebs = nba_stats_3[['REBOUNDS']]

# now return only the top 5
top_rebs = top_rebs.nlargest(5, 'REBOUNDS', keep="all")

# set the indexes to be columns
top_rebs.reset_index(inplace=True)

# rename the columns
top_rebs.rename(columns={"PLAYER_NAME": "player", "SEASON_ID": "season", "REBOUNDS": "total_rebounds"}, inplace=True)

# sort the dataframe
top_rebs.sort_values(['total_rebounds', 'player'], ascending=[False, True],inplace=True)

top_rebs

Your solution should match the dataframe below.

In [ ]:
top_rebs_soln = pd.read_csv('top_rebs.csv')
top_rebs_soln

### What are your questions on this exercise?

**Requirement**:  

Return a dataframe, top_mins, containing the player name for the top 10 average number of minutes for the 4 seasons together. 

    This means that we want to add up the total number of minutes the player has played and divide by the number of seasons, to get the average. Round the average to 1 decimal place, after sorting. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties).

Include the top 10 plus ties. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties). 

The dataframe should be sorted from most to least, with ties broken by name in reverse alphabetical order. Minutes are in the column MIN.

Use the nba_stats_3 dataframe as the input for this. 

The output dataframe should have the following columns:  `player`, `seasons_played`, `avg_minutes`.

#### What is our strategy for solving this problem?

0. Remember that the dataframe has SEASON_ID and PLAYER_NAME as indexes (see above).

1. Create two working dataframes, one to summarize minutes, the other to count seasons played.

2. With the average minutes dataframe:

    Compute the average minutes
    
    Rename the MINUTES column
    
    Set the indexes to be columns
    

2. With the seasons played dataframe:

    Compute the number of seasons each player played in
    
    Rename the MINUTES column
    
    Set the indexes to be columns
    
    
3. Merge the two dataframes

4. Rename the columns and keep only those columns required (two steps).

5. Keep only the top 10.

6. Drop the index column (created from the previous step)

7. Sort the dataframe.

8. Round the average minutes to one decimal place.

In [ ]:
# create the mins_df (working) dataframe with only the columns required
mins_df = nba_stats_3[['MINUTES']]

# compute the average minutes for each player
top_mins = mins_df.groupby("PLAYER_NAME").mean()
# rename the MINUTES column
top_mins.rename(columns={"MINUTES": "avg_minutes"}, inplace=True)
# set the index to be a column
top_mins.reset_index(inplace=True)

# compute how many seasons each played
num_seasons = mins_df.groupby(by=["PLAYER_NAME"]).count()
# rename the MINUTES column
num_seasons.rename(columns={"MINUTES": "seasons_played"}, inplace=True)
# set the indexes to be columns
num_seasons = num_seasons.reset_index()

# merge the two dataframes
top_mins = top_mins.merge(num_seasons, how='inner')

# rename the PLAYER_NAME column
top_mins.rename(columns={"PLAYER_NAME": "player"}, inplace=True)
# only keep the required columns
top_mins = top_mins[['player','seasons_played','avg_minutes']]

#now only keep the 10 highest, plus ties
top_mins = top_mins.nlargest(10, 'avg_minutes', keep="all").reset_index()
# drop the index column
del top_mins['index']

# sort the dataframe
top_mins.sort_values(['avg_minutes', 'player'], ascending=[False, False],inplace=True)

# round the avg_minutes column to one decimal place
top_mins = top_mins.round({'avg_minutes': 1})

top_mins

Your solution should match the dataframe below.

In [ ]:
top_mins_soln = pd.read_csv('top_mins.csv')
top_mins_soln

### What are your questions on this exercise?

### Now let's look at Boolean Masks.

#### What is a boolean mask? 

While boolean masks are typically used with numpy arrays, they can also be applied to pandas dataframes. 

We will introduce the concept here and later cover how they are used with numpy arrays, which is a bit different from how they are used with pandas. 

Conceptually, they are similar in pandas and numpy, but how each implements them is different.

**In pandas, a mask is used to filter and return only the rows that meet a certain condition.**

With pandas, we can use one of the comparison operators (`<, >, >=, <=, ==`), the `isin()` function, or the `contains()` function for strings.

Vanderplas has an EXCELLENT introduction to masks in his book, focused on numpy. Chapter linked to here:  https://jakevdp.github.io/PythonDataScienceHandbook/02.06-boolean-arrays-and-masks.html

In [ ]:
# mask to filter by comparison
minutes_mask = nba_stats['MINUTES'] >= 2000
minutes_mask

In [ ]:
# filter the dataframe using the mask
high_minutes = nba_stats[minutes_mask]
high_minutes

In [ ]:
# filter the dataframe directly, without creating the mask
# as a separate dataframe
high_minutes_2 = nba_stats[nba_stats['MINUTES'] >= 2000]
high_minutes_2

Note that the index returned is that of the index in the original dataframes. To reset the index to only those rows in the masked dataframe, use reset_index().

In [ ]:
# mask to filter by comparison
high_minutes_idx = nba_stats[minutes_mask].reset_index()
high_minutes_idx

In [ ]:
# mask using isin()
season_2017_mask = nba_stats['SEASON_ID'].isin([22017])
season_2017_mask

In [ ]:
# mask using isin(), with reset_index()
season_2017 = nba_stats[season_2017_mask].reset_index()
season_2017

In [ ]:
# mask using isin(), with reset_index()
season_2017_2 = nba_stats[nba_stats['SEASON_ID'].isin([22017])].reset_index()
season_2017_2

#### Now let's do a multiple comparison mask.

`Return the players with 2000 or more minutes in the 2017 and 2018 seasons.`

In [ ]:
# mask to filter by multiple comparison
multiple_mask = (nba_stats['MINUTES'] >= 2000) & (nba_stats['SEASON_ID'].isin([22017,22018]))
multiple_mask

In [ ]:
# return the dataframe
high_minutes_idx = nba_stats[multiple_mask].reset_index()
high_minutes_idx

#### Let's do a string comparison mask.

`Return all of the season stats for players named Anthony, in either their first or last names (or both).`

In [ ]:
name_anthony_mask = nba_stats['PLAYER_NAME'].str.contains('Anthony')
name_anthony_mask

In [ ]:
# mask to filter by string comparison
# return all of the players named Anthony
name_anthony = nba_stats[name_anthony_mask].reset_index()
name_anthony

#### What if we only wanted an array of the player's names, and not their season statistics?

Use the `unique()` function, which returns an array.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html

In [ ]:
name_anthony["PLAYER_NAME"].unique()

### Now let's do a more complex analysis, one that might be typical for a 2 or 3 point question on an exam.

**Requirement**: 

In the NBA, for a player to lead in any statistical category, he must have played in a minimum number of games. For a full season, that number is 58 games. If you are interested in a full explanation of the requirements, see the link below.

Write a function, `top_ten_scorers(df,min_games,season_id)` that returns the top 10 scoring leaders, in points per game, for any given season.

1. Return a dataframe, top_scorers, containing the player name for the top 10 average points per game for any season, for players who meet the minimum number of games qualification.

2. Round the average to 1 decimal place, after sorting. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties).

3. Include the top 10 plus ties. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties). 

4. The dataframe should be sorted from most to least, with ties broken by name in alphabetical order. 

5. The nba_stats dataframe will be the input for this, along with the season to be filtered for and minimum number of games to qualify. 

6. The output dataframe should have the following columns:  `player`, `games`, `points`, `PPG`.

https://www.nba.com/stats/help/statminimums

#### What is our strategy for solving this problem?

1. Create a dataframe with only the rows required, using a boolean mask.

2. Create the column `PPG`.

3. Keep only the required columns in the dataframe.

4. Rename the columns.

5. Return/keep only the top 10 for PPG in the dataframe, and reset the index.

6. Drop the index column.

7. Sort the dataframe.

8. Round PPG to one decimal place.

In [ ]:
def top_ten_scorers(df, min_games, season_id):
    ###
    ###YOUR CODE HERE
    
    # return a dataframe with players from the 2018 season who meet the 58 game minimum requirement
    # using boolean mask
    top_scorers = nba_stats[(nba_stats['GAMES_PLAYED'] >= min_games) & (nba_stats['SEASON_ID'].isin([season_id]))].reset_index()

    # # create the column PPG
    top_scorers['PPG'] = top_scorers['POINTS'] / top_scorers['GAMES_PLAYED']

    # # # keep only the required columns
    # top_scorers = top_scorers[['PLAYER_NAME','GAMES_PLAYED','POINTS','PPG']]

    # rename the columns
    top_scorers.rename(columns={"PLAYER_NAME": "player","GAMES_PLAYED": "games","POINTS": "points"}, inplace=True)
    # only keep the required columns
    top_scorers = top_scorers[['player','games','points','PPG']]

    # now only keep the 10 highest, plus ties
    top_scorers = top_scorers.nlargest(10, 'PPG', keep="all").reset_index()
    # drop the index column
    del top_scorers['index']

    # sort the dataframe
    top_scorers.sort_values(['PPG', 'player'], ascending=[False, True],inplace=True)

    # round the avg_minutes column to one decimal place
    top_scorers = top_scorers.round({'PPG': 1})

    return top_scorers

# test dataframe
top_scoring_players = top_ten_scorers(nba_stats,58,22018)
top_scoring_players

Your dataframe results should match those at this link:  https://www.espn.com/nba/stats/_/season/2019/seasontype/2

### What are your questions on this exercise, and on the notebook as a whole?

### Extra Credit, for fun (will not be covered during Bootcamp live session)

**Requirement**: 

In the NBA, the metric `PLUS_MINUS` provides a single number for the value of a player. The metric is defined as the difference between the number of points the player's team scores, minus the number of points the opposing team scores, during the time that the player is in the game.

A positive number means that, over the course of the season, the player's team scored that many more points than their opponents when he was on the court. Likewise for a negative number, his team scored that many fewer points.

In general, the best players have the highest `PLUS_MINUS`, and the worst player have the lowest `PLUS_MINUS`.

So let's see who the best and worst players were, during the 2020 season.

Return a dataframe, best_players, containing the top 10 players and their `PLUS_MINUS` value. Include the top 10 plus ties. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties). The dataframe should be sorted from most to least, with ties broken by name in reverse alphabetical order.  

Additionally, return a dataframe, worst_players, containing the bottom 10 players and their `PLUS_MINUS` value. Include the bottom 10 plus ties. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties). The dataframe should be sorted from lowest value to highest value, with ties broken by name in alphabetical order.

The output dataframes should have the following columns:  `PLAYER_NAME`, `PLUS_MINUS`. There is no need to rename the columns from their original names in the source dataframe for this exercise.

Use the nba_stats_2 dataframe as the input for this. 

The `nsmallest()` function is analogous to `nlargest` for finding the smallest values.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nsmallest.html





In [ ]:
# best players here
# keep only the 2020 season players
best_players = nba_stats[nba_stats['SEASON_ID'].isin([22020])].reset_index()

# # keep only the required columns
best_players = best_players[['PLAYER_NAME','PLUS_MINUS']]

# now only keep the 10 highest, plus ties
best_players = best_players.nlargest(10, 'PLUS_MINUS', keep="all").reset_index()
# drop the index column
del best_players['index']

# sort the dataframe
best_players.sort_values(['PLUS_MINUS', 'PLAYER_NAME'], ascending=[False, True],inplace=True)

best_players

Your solution should match the dataframe below.

In [ ]:
best_players_soln = pd.read_csv('best_players.csv')
best_players_soln

In [ ]:
# worst players here
# keep only the 2020 season players
worst_players = nba_stats[nba_stats['SEASON_ID'].isin([22020])].reset_index()

# # keep only the required columns
worst_players = worst_players[['PLAYER_NAME','PLUS_MINUS']]

# now only keep the 10 highest, plus ties
worst_players = worst_players.nsmallest(10, 'PLUS_MINUS', keep="all").reset_index()
# drop the index column
del worst_players['index']

# sort the dataframe
worst_players.sort_values(['PLUS_MINUS', 'PLAYER_NAME'], ascending=[True, True],inplace=True)

worst_players

Your solution should match the dataframe below.

In [ ]:
worst_players_soln = pd.read_csv('worst_players.csv')
worst_players_soln